# Bayes Classifier and Maximum Likelihood for a multivariate Gaussian density
# Classificateur de Bayes et maximum de vraisemblance d'une loi gaussienne multivariée

## Preface/Préface

Begin, if necessary, by recalling the course notes on the [Bayes classifier](https://studium.umontreal.ca/pluginfile.php/4027948/mod_resource/content/4/7_bayes_classifier-en.pdf) and the principle of [maximum likelihood](https://studium.umontreal.ca/pluginfile.php/4003963/mod_resource/content/4/5_gaussian_distribution_en.pdf).

Remember that you need to **submit** a `.py` file in Gradescope (and **not** a jupyter notebook). You can either copy the code from the notebook's cells and paste it into a new file, or use Jupyter's tool to export to a `.py` file (File / Download as / Python). **The name of your submission HAS TO BE `solution.py`**. For the autograder to work properly, you also need to comment all `print` statements.

<hr>

Commencez, si nécessaire, par faire une révision des notes de cours portant sur le [classificateur de Bayes](https://studium.umontreal.ca/pluginfile.php/4027948/mod_resource/content/4/7_bayes_classifier-en.pdf) et sur le principe du [maximum de vraisemblance](https://studium.umontreal.ca/pluginfile.php/4003963/mod_resource/content/4/5_gaussian_distribution_en.pdf).

Souvenez-vous que vous devez **soumettre** un fichier `.py` sur Gradescope (et **non** un jupyter notebook). Vous pouvez soit copier le code des cellules du jupyter notebook et le coller dans un nouveau fichier `.py` ou utiliser la fonctionalité de jupyter notebook afin d'exporter le fichier en un fichier `.py` (File / Download as / Python). **Le nom du fichier que vous allez soumettre DOIT ÊTRE** `solution.py`. Pour que le correcteur automatique fonctionne correctement, vous devez commenter toutes les assertions `print` dans votre `.py`.






## High level description/Description détaillée

Today we are going to build a **multi-class Bayes classifier**. This means that instead of modeling $ p (\mbox{class} \ | \ \mbox{example}) $ (or $ p (y \ | \ x) $), we will instead use the Bayes equation

$$ p (\mbox{class} \ | \ \mbox{example}) = \frac{p (\mbox{example} \ | \ \mbox{class}) p (\mbox {class})} {\sum_{c'= 1}^{m} p_\mbox{c'}(x) P_\mbox{c'}} $$

and model the different pieces. In fact, we just need to model the numerator since the denominator is a normalization constant. In addition, $ P_\mbox{c '} = n_c / n $

The term $ p (\mbox{class}) $ represents the prior probability of a class, that is, our a priori belief - before we have seen a particular example - about the probability that an unknown example belongs to this class. We will represent this belief a priori for a class by the frequency of the latter in the training data: $ \frac{n_c}{n} $ where $ n_c $ = number of examples of the class $ c $, and $ n $ = number of training examples.

We will use **multivariate Gaussian densities** to model the different $ p (\mbox{example} \ | \ \mbox{class}) $. This means that **for each class** $c$, we will assume that the "true" distribution $ p (\mbox{example} \ | \ \mbox{class}) $ has the form of a multivariate Gaussian for which we will try to learn the parameters $ \mu_c $ and $ \Sigma_c $. In practice, we will limit ourselves today to a particular case of this distribution: the **isotropic Guassian**, i.e. the covariance matrix $ \Sigma_c $ of each Gaussian is diagonal and that each element of this diagonal is the same ( we denote it `sigma_sq`, in order to represent the variance $ \sigma^2 $). Thus we have a single parameter to control the expression of the covariance for each class. It's easier (for us and for the computer) to calculate, but it also means that our model is less powerful.

Hence, we have a very simple parametric model for each class $c$. The parameters are the average $ \mu_c $ (a vector of the same dimension as the dimension of the system input) and the variance $ \sigma^2_c $ (a single scalar in our simple model, which will multiply the identity matrix). We are going to learn this model with the **maximum likelihood principle**. For each class, we will find the values of the parameters that maximize the log-likelihood of the training data from this class:

$$ \log \prod_i^n p(X = x_i) $$

For an isotropic Gaussian of dimension $d$, the maximum likelihood estimators of $\mu$ and $\sigma^2$ are given by: 

$$\mu_{ML} = \frac{1}{n} \sum_{i=1}^{n} x_i$$

$$\sigma_{ML}^2 = \frac{1}{nd} \sum_{i=1}^{n} (x_i-\mu_{ML})^T(x_i-\mu_{ML})$$

Having found the parameters that maximize the likelihood for each class, we can calculate each $ p (\mbox{example} \ |  \ \mbox{class}) $. It is now sufficient to apply the Bayes rule in order to classify a new example. More precisely, we want to choose, for an example, the class that maximizes $ p(\mbox{example} \ | \ \mbox{class}) p(\mbox{class}) $ or, equivalently, $ \log (p (\mbox{example } \ | \ \mbox{class} ) p(\mbox{class})) $. **At test time, our classifier needs to evaluate this quantity for each class, and use the obtained values to assign a class label**.

<hr>

Aujourd'hui, nous allons construire un **classîficateur de Bayes multi-classes**. Ceci veut dire qu'à la place de modéliser  $ p (\mbox{classe} \ | \ \mbox{exemple}) $ (ou $ p (y \ | \ x) $), nous allons plutôt utiliser la formule de Bayes suivante: 

$$ p (\mbox{classe} \ | \ \mbox{exemple}) = \frac{p (\mbox{exemple} \ | \ \mbox{classe}) p (\mbox {classe})} {\sum_{c'= 1}^{m} p_\mbox{c'}(x) P_\mbox{c'}} $$

et allons modéliser différents termes. En fait, nous avons uniquement besoin de modéliser le numérateur puisque le dénominateur  est une constante de normalisation. De plus, $ P_\mbox{c '} = n_c / n $.

Le terme $ p (\mbox{classe}) $ représente la probabilité à priori d'une classe, c'est-à-dire notre hypothèse à priori - avant avoir vu un certain exemple - de la probabilité qu'un exemple quelconque appartienne à cette classe. Nous allons représenter cette hypothèse par sa fréquence dans notre jeu d'entraînement: $ \frac{n_c}{n} $, où $ n_c $ est le nombre d'exemples appartement à cette classe et $ n $ est le nombre d'exemples dans notre jeu d'entraînement.

Nous allons utiliser des **gaussiennes multivariées** afin de modéliser les différentes probabilités $ p (\mbox{exemple} \ | \ \mbox{classe}) $. En d'autres mots, nous assumons que pour **chaque classe** $c$, la *vraie* distribution $ p (\mbox{exemple} \ | \ \mbox{classe}) $  possède la forme d'une loi gaussienne multivarié pour laquelle nous allons apprendre les paramètres $ \mu_c $ et $ \Sigma_c $. Nous allons nous limiter aujourd'hui à un cas particulier de cette distribution, soit la loi **gaussienne isotropique**. Plus précisément, la matrice de covariance $ \Sigma_c $ de chacune de ces distributions est une matrice diagonale dont chacun des éléments sur la diagonale sont égaux (nous notons `sigma_sq` la variance $ \sigma^2 $). Ainsi, nous n'avons qu'un seul paramètre à estimer afin de déterminer la matrice de covariance pour chaque classe. C'est certainement plus facile (pour nous et pour l'ordinateur) de calculer sa valeur, mais ceci veut également dire que notre modèle possède moins de capacité.

Nous avons alors un modèle paramétrique assez simple pour chaque classe $c$. Les paramètres sont l'espérance $ \mu_c $ (un vecteur de la même dimension que nos valeurs d'entrées) et la variance $ \sigma^2_c $ (un scalaire multiplié par la matrice identité). Nous allons apprendre (estimer) ces paramètres à l'aide du maximum de vraisemblance. Pour chaque classe, nous trouverons les valeurs des paramètres qui maximisent la *log-vraisemblance* des points d'entraînement de cette classe: 

$$ \log \prod_i^n p(X = x_i) $$

Pour une gaussienne isotropique de dimension $d$, les estimateurs du maximum de vraisemblance $\mu$ et $\sigma^2$ sont:
 

$$\mu_{ML} = \frac{1}{n} \sum_{i=1}^{n} x_i$$

$$\sigma_{ML}^2 = \frac{1}{nd} \sum_{i=1}^{n} (x_i-\mu_{ML})^T(x_i-\mu_{ML})$$

Après avoir trouvé les paramètres qui maximisent la vraisemblance pour chaque classe, nous pouvons ensuite calculer chaque probabilité $ p (\mbox{exemple} \ |  \ \mbox{classe}) $. Il est maintenant suffisant d'appliquer la règle de Bayes afin de classer un nouvel exemple. Plus précisément, nous voulons trouver, pour un exemple donné, la classe qui maximise $ p(\mbox{exemple} \ | \ \mbox{classe}) p(\mbox{classe}) $ (ou de manière équivalente $ \log (p (\mbox{exemple } \ | \ \mbox{classe} ) p(\mbox{classe})) $. **Au moment test, notre classifieur doit évaluer cette quantité pour chaque classe et utiliser ces valeurs afin d'assigner une classe à l'exemple.**


## Code to be completed/Code à compléter

For the `GaussianMaxLikelihood` class:
 
  - **Calculate** mu ($\mu$) and sigma_sq ($ \sigma^2 $), the mean and the variance in `GaussianMaxLikelihood.train`. 
  - **Calculate** the value of the Gaussian density function in `GaussianMaxLikelihood.loglikelihood`, store it in the variable `log_prob` and **return it**.
  
In `__init__`, we initialize `self.mu` and `self.sigma_sq`. But both variables will be updated when the `train` function is called.

<hr>

Pour la classe `GaussianMaxLikelihood`:

  - **Calculez** mu ($\mu$) et sigma_sq ($ \sigma^2 $), la moyenne et la variance dans `GaussianMaxLikelihood.train`.
  - **Calculez** la densité de la loi gaussienne dans `GaussianMaxLikelihood.loglikelihood`, definissez la comme étant la variable `log_prob` et **retournez la**.
  
Dans `__init__`, nous initialisons `self.mu` et `self.sigma_sq`. Ces variables seront ensuite modifiées lorsque la fonction `train` sera appelée. 


In [0]:
import numpy as np

In [0]:
class GaussianMaxLikelihood:
    def __init__(self, n_dims):
        self.n_dims = n_dims
        self.mu = np.zeros(n_dims)
        # We only save a scalar standard deviation because our model is the isotropic Gaussian
        # We avons un scalaire comme écart-type car notre modèle est une loi gaussienne isotropique
        self.sigma_sq = 1.0

    # For a training set, the function should compute the ML estimator of the mean and the variance
    # Pour un jeu d'entraînement, la fonction devrait calculer les estimateur ML de l'espérance et de la variance
    def train(self, train_data):
        # Here, you have to find the mean and variance of the train_data data and put it in self.mu and self.sigma_sq
        # Ici, nous devons trouver la moyenne et la variance dans train_data et les définir dans self.mu and self.sigma_sq
        
        # ---> WRITE CODE HERE/ÉCRIVEZ VOTRE CODE ICI
        self.mu = 
        self.sigma_sq = 
        
    # Returns a vector of size nb. of test ex. containing the log probabilities of each test example under the model.
    # Retourne un vecteur de dimension égale au nombre d'ex. test qui contient les log probabilité de chaque 
    # exemple test
    def loglikelihood(self, test_data):

        # comment the following line once you have completed the calculation of log_prob
        # mettez en commentaire cette ligne lorsque vous avez complétez le calcul de log_prob

        # the following line calculates log(normalization constant)
        # la ligne suivante calcule le log(normalization constant)
        c = - ((self.n_dims / 2) * np.log(2 * np.pi)) - ( self.n_dims * np.log(np.sqrt(self.sigma_sq)) )
        # It is necessary to calculate the value of the log-probability of each test example
        # under the model determined by mu and sigma_sq. The vector of probabilities is / will be log_prob
        # Il est nécessaire de calculer la log-probabilité de chaque exemple test sous le modèle déterminé
        # par mu et sigma_sq. Le vecteur de probabilité est/sera log_prob
        
        # ---> WRITE CODE HERE/ÉCRIVEZ VOTRE CODE ICI
        # log_prob = 
        return log_prob

SyntaxError: ignored

For class `BayesClassifier`:

  - **Complete** `bayes_classifier.loglikelihood`
  
`self.maximum_likelihood_models` is a list of size equal to the number of classes. Each element is a model that allows log likelihood evaluations using a `loglikelihood` method.

`self.priors` is a list of scalars. The numbers represent the prior probabilities of the classes.

<hr>

Pour la classe `BayesClassifier`:

  - **Complétez** `bayes_classifier.loglikelihood`
  
`self.maximum_likelihood_models` est une liste de longueur égale au nombre de classes. Chaque élément est un modèle qui permet de calculer les log-vraisemblances en utilisant la méthode `loglikelihood`.

`self.priors` est une liste de scalaires. Ils représentent les probabilités à priori des classes.

In [0]:
class BayesClassifier:
    def __init__(self, maximum_likelihood_models, priors):
        self.maximum_likelihood_models = maximum_likelihood_models
        self.priors = priors
        if len(self.maximum_likelihood_models) != len(self.priors):
            print('The number of ML models must be equal to the number of priors!')
        self.n_classes = len(self.maximum_likelihood_models)

    # Returns a matrix of size number of test ex. times number of classes containing the log
    # probabilities of each test example under each model, trained by ML.
    # Retourne une matrice de dimension [nb d'ex. test, nb de classes] contenant les log
    # probabilités de chaque ex. test sous le modèle entrainé par le MV.
    def loglikelihood(self, test_data):

        log_pred = np.zeros((test_data.shape[0], self.n_classes))

        for i in range(self.n_classes):
            # Here, we will have to use maximum_likelihood_models[i] and priors to fill in
            # each column of log_pred (it's more efficient to do a entire column at a time)
            # Ici, nous devrons utiliser maximum_likelihood_models[i] et priors pour remplir
            # chaque colonne de log_pred (c'est plus efficace de remplir une colonne à la fois) 
            
            # ---> WRITE CODE HERE/ÉCRIVEZ VOTRE CODE ICI
            log_pred[:, i] 

        return log_pred

In this part, we will train a BayesClassifier on the iris dataset, according to the what we described in the High level description section. We will use the classes BayesClassifier and GaussianMaxLikelihood.

Contrary to what we did in the previous labs, **we will not use a train/test set**. We will simply use the whole dataset to train the classifier, and we are going to ask the trained classifier to **make predictions on the training set itself**. 

There are extra questions at the end of this notebook where you can play around and use a proper train/test split. But this will not be tested by gradescope.

**Complete** the function `get_accuracy(test_inputs, test_labels)`, that takes as input a test set, and returns the trained classifier's (`classifier`) accuracy on this test set. Your function should return a number between 0 and 1.

To make sure everything works fine, you can test your code by calling your function on the train dataset (This part of the code after the function is provided for you). But remember that your function should work with any inputs, and not only (iris_train, iris_labels).

<hr>

Dans cette partie, nous allons entraîner BayesClassifier sur le dataset iris, tel qu'expliqué dans la description détaillée. Nous allons utiliser les classes BayesClassifier et GaussianMaxLikelihood.

Contrairement à ce que nous avons fait dans le lab précédent, **nous n'allons pas utiliser un dataset train/test**. Nous allons simplement utiliser le dataset complet pour entraîner le modèle et ensuite **faire des prédictions sur le training set lui-même.**

Des questions bonus se trouvent à la fin de ce notebook pour que vous puissiez explorer avec un ensemble d'entraînement/test. Ces questions ne seront cependant pas testées sur gradescope.

**Complétez** la fonction `get_accuracy(test_inputs, test_labels)`, qui prend en paramètre un ensemble test, et retourne l'exactitude du classifieur (`classifier`) entraîné. Votre fonction devrait retourner un nombre entre 0 et 1.

Pour vous assurer que tout fonctionne, vous pouvez tester votre code par appeller votre fonction sur l'ensemble d'entraînement (cette partie du code, après la fonction, vous est donnée). Souvenez-vous cependant que votre fonction devrait fonctionner avec toutes données d'entrée, et pas seulement (iris_train, iris_labels).

In [0]:
# We load the dataset and split it into input data, and labels
# On télécharge le dataset et le séparons en input et en étiquettes
import sys
import numpy as np
IN_COLAB = 'google.colab' in sys.modules
if IN_COLAB:
    iris = np.loadtxt('http://www.iro.umontreal.ca/~dift3395/files/iris.txt')
else:
    iris = np.loadtxt('iris.txt')
iris_train = iris[:, :-1]
iris_labels = iris[:, -1]

# We split the input data into three sub-datasets, corresponding to the classes 1, 2, and 3. 
# It is necessary to make such a split, in order to train a GaussianMaxLikelihood model per class
# Note that the split (0:50, 50:100, 100:150) is not arbitrary. It corresponds to the three classes in the dataset
# On sépare le input en trois sous-ensembles, correspondant aux classes 1, 2 et 3. Il est nécessaire
# de faire cette partition afin d'entraîner GaussianMaxLikelihood pour chaque classe. Notez que la partition
# (0:50, 50:100 et 100:150) n'est pas arbitraire: elle correspond aux trois classes dans le dataset.
iris_train1 = iris_train[0:50, :]
iris_train2 = iris_train[50:100, :]
iris_train3 = iris_train[100:150, :]

# We create a model per class (using maximum likelihood), and train each of them using the corresponding data
# Nous créons un modèle par classe (en utilisant le maximum de vraisemblance) et les entraînons
model_class1 = GaussianMaxLikelihood(4)
model_class2 = GaussianMaxLikelihood(4)
model_class3 = GaussianMaxLikelihood(4)
model_class1.train(iris_train1)
model_class2.train(iris_train2)
model_class3.train(iris_train3)

# We create a list of all our models, and the list of prior values
# Here the priors are calculated exactly because we know the number of representatives per class.
# Nous créons une liste de nos modèles et une liste de nos priors. Ici, nos priors sont
# déjà calculés car nous savons les proportions par classe
model_ml = [model_class1, model_class2, model_class3]
priors = [1./3, 1./3, 1./3]

# We create our classifier with our list of Gaussian models and our priors
# Nous créons notre classifieur avec notre liste de modèles gaussiens et nos prob à priori
classifier = BayesClassifier(model_ml, priors)

# Returns a number between 0 and 1 representing the accuracy of the model on the test_inputs
# Retourne un nombre entre 0 et 1 représentant l'exactitude du modèle sur les test_inputs
def get_accuracy(test_inputs, test_labels):
    # We can calculate the log-probabilities according to our model
    # Nous pouvons calculez les log-probabilités selon notre modèle
    
    # ---> WRITE CODE HERE/ÉCRIVEZ VOTRE CODE ICI

    # It now remains to calculate the predicted labels
    # Il reste à calculer les classes prédites
    
    # ---> WRITE CODE HERE/ECRIVEZ VOTRE CODE ICI
    
    # Return the accuracy by comparing your predicted labels to the actual labels
    # Retournez l'exactitude en comparant les classes prédites aux vraies étiquettes

    # ---> WRITE CODE HERE/ECRIVEZ VOTRE CODE ICI
    return acc
    
if __name__=='__main__':
    print("The training accuracy is : {:.1f} % ".format(100 * get_accuracy(iris_train, iris_labels)))

## Once you're done/Une fois terminé (Bonus)

- Change your code so that `GaussianMaxLikelihood` calculates 1) a diagonal covariance matrix (where we estimate the variance for each component / trait of the input) 2) a full covariance matrix. You can for example have a parameter `cov_type` that can be "isotropic", "diagonal" or "full". The `numpy.cov` and` numpy.var` commands will probably be useful.

- Instead of using the whole iris dataset to train a `BayesClassifier`, peform a random split of the data (2/3, 1/3) to obtain a train set and a test set, and use the train set to train your classifier. Compare the obtained accuracies on the train set and the test set. Remember to adjust the code that defines `iris_train1`, `iris_train2`, `iris_train3`, and `priors` accordingly. Compare the train and test accuracies of 3 models: one that uses "isotropic" gaussian models, one that uses "diagonal" gaussian models, and one that uses "full" gaussian models.

- Instead of using the 4 features to train the classifier, use only 2 and try to make nice visualizations as we did in Lab 2. You can of course use the utility functions from there to make your plots.

<hr>

- Changez votre code afin que `GaussianMaxLikelihood` calcule 1) une matrice de variance-covariance diagonale (où on estime la variance de chaque élément/trait des données d'entrée) 2) une matrice de variance-covariance complète. Vous pouvez par exemple avoir un paramètre `cov_type` qui peut être `isotropic`, `diagonal` ou `full`. Les commandes `numpy.cov` et `numpy.var` peuvent être utiles.

- À la place d'utiliser le dataset complet iris pour entraîner `BayesClassifier`, performez une séparation aléatoire de (2/3, 1/3) afin d'obtenir un ensemble d'entraînement et un ensemble test. Comparez les exactitudes du classifieur obtenues à partir des ensembles train et test. Prenez soin d'ajuster le code qui définit `iris_train1`, `iris_train2`, `iris_train3` et `priors` en conséquence. Comparez l'exactitude des ensembles train et test des 3 modèles: gaussienne isotropique, gaussienne diagonale et gaussienne *complète*.

- Plutôt que d'utiliser les 4 traits afin d'entraîner votre classifieur, utilisez seulement que deux caractéristiques et effectuer des analyses de visualisation telles que faites dans le lab 2. Vous pouvez biensûr faire usages des fonctions du lab 2 pour faire vos graphiques.